In [50]:
import os
import json
import traceback
import pandas as pd
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import PromptTemplate

from dotenv import load_dotenv

load_dotenv()

True

In [51]:
KEY = os.getenv("GROQ_API_KEY")

In [53]:
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=KEY)

In [54]:
NUMBER = 5
TONE = "simple"
SUBJECT = "Machine Learning"

In [59]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    },
    "2":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    },
    "3":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    },
    "4":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    },
    "5":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    }
}

In [60]:
RESPONSE_JSON = json.dumps(RESPONSE_JSON)
RESPONSE_JSON

'{"1": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice question", "option": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [61]:
FILE_PATH = "D:/AI Projects/MCQs Generator/data.txt"

In [63]:
with open(FILE_PATH, "r") as f:
    TEXT = f.read()

In [64]:
template = """
Text: {text}
You are an expert mcq maker. Given the above text, it is yor job to \
create a quiz of {number} multiple choice questions for {subject} in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON and use it as a guide. \
Ensure to make {number} MCQs.
###  RESPONSE_JSON
{response_json} 

"""

Template1 = PromptTemplate.from_template(template)

In [65]:
Template1

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText: {text}\nYou are an expert mcq maker. Given the above text, it is yor job to create a quiz of {number} multiple choice questions for {subject} in {tone} tone.\nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like RESPONSE_JSON and use it as a guide. Ensure to make {number} MCQs.\n###  RESPONSE_JSON\n{response_json} \n\n')

In [81]:
input_dict = {
    'number': NUMBER,
    'response_json': RESPONSE_JSON,
    'subject': SUBJECT,
    'text': TEXT,
    'tone': TONE
}

In [83]:
quiz_generation = Template1 | llm | StrOutputParser()
quiz = quiz_generation.invoke(input_dict)

In [86]:
quiz
with open("MCQs.txt", "w") as f:
    f.write(quiz)

In [77]:
template2 = """
You are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
if the quiz is not at per with the cognitive and anlytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer from the above quiz:
Make sure to format the quiz like RESPONSE_JSON and use it as a guide. \
###  RESPONSE_JSON
{response_json}
"""
Template2 = PromptTemplate.from_template(template2)

In [69]:
def add_quiz_to_input(quiz:str):
    return{**input_dict,"quiz":quiz}

In [74]:
chain = Template1 | llm | StrOutputParser() | RunnableLambda(add_quiz_to_input)| Template2 | llm | StrOutputParser()

In [78]:
quiz_evaluation = chain.invoke(input_dict)

In [79]:
quiz_evaluation

'### Complexity: Moderate\n\n### RESPONSE_JSON\n{"analysis": "The quiz is well-structured, but some questions may be too specific or require prior knowledge of machine learning history. To better fit the students\' abilities, the questions should be more focused on fundamental concepts and general knowledge of machine learning.", \n"updated_quiz": {\n"1": {"mcq": "What is the primary goal of machine learning?", "option": {"a": "To classify data", "b": "To make predictions", "c": "To improve performance with experience", "d": "All of the above"}, "correct": "d"},\n"2": {"mcq": "Which of the following is a key characteristic of machine learning?", "option": {"a": "Ability to think like humans", "b": "Ability to recognize patterns", "c": "Ability to learn from experience", "d": "Ability to analyze data"}, "correct": "c"},\n"3": {"mcq": "What is required for a computer program to be said to learn from experience, according to Tom M. Mitchell?", "option": {"a": "Its performance at tasks imp

In [87]:
with open("MCQs.txt", "a") as f:
    f.write(quiz_evaluation)